In [73]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [74]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

data_table = soup.findAll('table')[0]

column_headers = [th.getText().strip('\n') for th in data_table.findAll('tr', limit=1)[0].findAll('th')]

data_rows = data_table.findAll('tr')[1:]

data = [[td.getText().strip('\n').strip(']') for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]

df = pd.DataFrame(data, columns=column_headers)
df = df[(df['Borough'] != 'Not assigned')]

df['Neighbourhood'] = df.apply(lambda row: row['Borough'] if row['Neighbourhood'] == 'Not assigned' else row['Neighbourhood'],axis=1)

toronto_data = df.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]).reset_index()

toronto_data.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

toronto_data.head(n=10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [75]:
toronto_data.shape

(103, 3)